In [ ]:
! pip install transformers
! pip install torch
! pip install torchaudio
#! pip install tf-keras

In [2]:
import os
import json
import torch
import\
    torchaudio
from glob import glob

from torch.utils.data import Dataset
from torch.utils.data import DataLoader

from transformers import WhisperForConditionalGeneration, WhisperProcessor, TrainingArguments, Trainer, AdamW, get_scheduler

In [3]:
# ----- Params -----

# 라벨 디렉터리
# data_dir = "datasets/OldPeople_Voice/label/"
data_dir = "E:\\139-1.중·노년층 한국어 방언 데이터 (강원도, 경상도)\\01-1.정식개방데이터\\Training\\02.라벨링데이터\\"
# 오디오 디렉터리
audio_dir = "E:\\139-1.중·노년층 한국어 방언 데이터 (강원도, 경상도)\\01-1.정식개방데이터\\Training\\01.원천데이터\\"
# 학습된 데이터
save_dir = "whisper_finetuned"

# Validation label data
validation_label = "E:\\139-1.중·노년층 한국어 방언 데이터 (강원도, 경상도)\\01-1.정식개방데이터\\"
# Validation audio data
validation_audio = "E:\\139-1.중·노년층 한국어 방언 데이터 (강원도, 경상도)\\01-1.정식개방데이터\\"

# ----- ------ -----

In [4]:
class CustomAudioDataset(Dataset):
    def __init__(self, json_list, processor):
        self.processor = processor
        self.data = []

        # 모든 JSON 파일을 리스트로
        for json_path in json_list:
            with open(json_path, "r", encoding="utf-8") as f:
                data = json.load(f)
            
            # 오디오 파일 경로
            audio_file = os.path.join(audio_dir, data["fileName"]+".wav")
            
            # 파일이 실제 존재하는지 확인 (오류 방지)
            if not os.path.exists(audio_file):
                print(f"⚠️ Warning: {audio_file} 파일이 존재하지 않습니다.")
                continue  # 해당 파일 건너뛰기
            
            text = data["transcription"]["standard"]
            self.data.append((audio_file, text))

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        audio_file, text = self.data[idx]

        waveform, sample_rate = torchaudio.load(audio_file)

        # 16kHz 샘플링 for Whisper
        if sample_rate != 16000:
            waveform = torchaudio.transforms.Resample(sample_rate, 16000)(waveform)

        # 오디오 데이터 변환
        input_features = self.processor(
            waveform.squeeze(0).numpy(),
            sampling_rate=16000,
            return_tensors="pt"
        ).input_features

        # 텍스트 토큰화 하기
        labels = self.processor.tokenizer(text, return_tensors="pt").input_ids

        return {
            "input_features": input_features.squeeze(0),
            "labels": labels.squeeze(0)
        }

In [5]:
def load_data(json_path):
    with open(json_path, "r", encoding="utf-8") as f:
        data = json.load(f)

    audio_file = os.path.join(audio_dir, data["fileName"]+".wav")
    text = data["transcription"]["standard"]

    return {
        "audio": audio_file,  # 파일 경로 저장
        "text": text,
        # "duration": duration
    }

In [6]:
batch_size = 4
learning_rate = 1e-5
num_epochs = 3
gradient_accumulation_steps = 2
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

cuda


In [7]:
json_list = glob(f"{data_dir}**/*.json", recursive=True)
# print(sorted(json_list)[:10])
print("data 수",len(json_list))

data 수 303149


In [8]:
# JSON file 검사
# 라벨 데이터에 문제 발생 시... 직접 수정 바랍니다.
# 139-1.중·노년층 한국어 방언 데이터 (강원도, 경상도)\01-1.정식개방데이터\Training\02.라벨링데이터\TL_02. 경상도_01. 1인발화 따라말하기\st_set1_collectorgs100_speakergs442_54_10 에서 , 중복 문제 있었음!
json_path = ""
try:
    for json_path in json_list:
        with open(json_path, "r", encoding="utf-8") as f:
            json.load(f)
    print("json 파일 데이터 무결성 검사 끝")
except json.JSONDecodeError as e:
    print(json_path,", label data 오류발생!!!")

KeyboardInterrupt: 

In [13]:
model = WhisperForConditionalGeneration.from_pretrained("SungBeom/whisper-small-ko")
processor = WhisperProcessor.from_pretrained("SungBeom/whisper-small-ko")

model.to(device)

WhisperForConditionalGeneration(
  (model): WhisperModel(
    (encoder): WhisperEncoder(
      (conv1): Conv1d(80, 768, kernel_size=(3,), stride=(1,), padding=(1,))
      (conv2): Conv1d(768, 768, kernel_size=(3,), stride=(2,), padding=(1,))
      (embed_positions): Embedding(1500, 768)
      (layers): ModuleList(
        (0-11): 12 x WhisperEncoderLayer(
          (self_attn): WhisperSdpaAttention(
            (k_proj): Linear(in_features=768, out_features=768, bias=False)
            (v_proj): Linear(in_features=768, out_features=768, bias=True)
            (q_proj): Linear(in_features=768, out_features=768, bias=True)
            (out_proj): Linear(in_features=768, out_features=768, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=768, out_features=3072, bias=True)
          (fc2): Linear(in_features=3072, out_features=768, bias=True)
        

In [10]:
dataset = CustomAudioDataset(json_list, processor)
print(f"{len(dataset)}개 로드.")
train_dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True, collate_fn=lambda x: x)
print(f"steps 총 {len(train_dataloader)}")

303149개 로드.


In [11]:
# optimizer & scheduler 
optimizer = AdamW(model.parameters(), lr=learning_rate)
lr_scheduler = get_scheduler(
    "linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=len(train_dataloader) * num_epochs
)

C:\Users\haiiron\.conda\envs\cuda_38\lib\site-packages\transformers\optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [ ]:
error_count = 0  # 에러 발생 횟수 저장

# Train
for epoch in range(num_epochs):
    model.train()
    total_loss = 0

    for step, batch in enumerate(train_dataloader):
        try:
            print("try ", step)
            # 배치에서 input_features와 labels 추출
            input_features = [item["input_features"].to(device) for item in batch]
            labels = [item["labels"].to(device) for item in batch]

            # padding 처리
            input_features = torch.nn.utils.rnn.pad_sequence(input_features, batch_first=True, padding_value=0)
            labels = torch.nn.utils.rnn.pad_sequence(labels, batch_first=True, padding_value=-100)

            # 모델에 입력
            outputs = model(input_features, labels=labels)
            loss = outputs.loss / gradient_accumulation_steps  # gradient accumulation 적용
            loss.backward()

            if (step + 1) % gradient_accumulation_steps == 0 or (step + 1 == len(train_dataloader)):
                optimizer.step()
                lr_scheduler.step()
                optimizer.zero_grad()

            total_loss += loss.item()

        except Exception as e:
            error_count += 1
            print(f"️ Error at step {step}: {e}")
            continue  # 에러 발생 시 건너뛰기

    avg_loss = total_loss / (len(train_dataloader) - error_count)
    print(f"🚀 Epoch [{epoch+1}/{num_epochs}], Loss: {avg_loss:.4f}, Errors: {error_count}")

    # 모델 저장
    model.save_pretrained(f"{save_dir}/epoch_{epoch+1}")
    processor.save_pretrained(f"{save_dir}/epoch_{epoch+1}")


C:\Users\haiiron\.conda\envs\cuda_38\lib\site-packages\transformers\models\whisper\modeling_whisper.py:545: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:555.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(
Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.43.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


⚠️ Error at step 89: Labels' sequence length 488 cannot exceed the maximum allowed length of 448 tokens.
⚠️ Error at step 180: Labels' sequence length 463 cannot exceed the maximum allowed length of 448 tokens.
⚠️ Error at step 222: Labels' sequence length 498 cannot exceed the maximum allowed length of 448 tokens.
⚠️ Error at step 251: Labels' sequence length 492 cannot exceed the maximum allowed length of 448 tokens.
⚠️ Error at step 285: Labels' sequence length 512 cannot exceed the maximum allowed length of 448 tokens.
⚠️ Error at step 329: Labels' sequence length 453 cannot exceed the maximum allowed length of 448 tokens.
⚠️ Error at step 348: Labels' sequence length 450 cannot exceed the maximum allowed length of 448 tokens.
⚠️ Error at step 480: Labels' sequence length 559 cannot exceed the maximum allowed length of 448 tokens.
⚠️ Error at step 587: Labels' sequence length 494 cannot exceed the maximum allowed length of 448 tokens.
⚠️ Error at step 614: Labels' sequence length 4

# TEST

In [ ]:
from transformers import WhisperForConditionalGeneration, WhisperProcessor

model_path = "whisper_finetuned/epoch_1"  # X는 저장한 에포크 번호
model = WhisperForConditionalGeneration.from_pretrained(model_path)
processor = WhisperProcessor.from_pretrained(model_path)

device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

# 테스트 오디오 파일
audio_file = audio_dir + "노인남여_노인대화77_F_김XX_62_제주_실내_84051.WAV"

# 오디오 파일 로드
waveform, sample_rate = torchaudio.load(audio_file)

# Whisper는 16kHz 샘플링 속도를 사용하므로 변환 필요
if sample_rate != 16000:
    waveform = torchaudio.transforms.Resample(sample_rate, 16000)(waveform)

# 모델의 입력으로 변환
input_features = processor(waveform.squeeze(0).numpy(), sampling_rate=16000, return_tensors="pt").input_features
input_features = input_features.to(device)

# 모델을 통해 예측 수행
with torch.no_grad():
    predicted_ids = model.generate(input_features)

# 예측된 텍스트 디코딩
transcribed_text = processor.decode(predicted_ids[0], skip_special_tokens=True)

print("예측된 텍스트:", transcribed_text)
